Подлючение необходимых библиотек.

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.estimators import BDeuScore, K2Score, BicScore
from pgmpy.estimators import PC,HillClimbSearch
from pomegranate import *
import seaborn as sns
from sklearn import *
from sklearn.preprocessing import KBinsDiscretizer
import time

<h2>Подготовка данных</h2>

Считывание изначального датасета.

In [ ]:
geo_data = pd.read_excel('dataset_geologist.xlsx')
geo_data.head()

Считывание датасета со значениями пропущенных в датасете geo_data данных.

In [ ]:
mis_data = pd.read_csv('missimg_values.csv', sep=';')
mis_data.head()

Составление тестовой выборки из двух предыдущих.

In [ ]:
test_data = geo_data.copy()
table = mis_data[['Unnamed: 0']].copy()
size_of_table = len(table)
i = 0

while size_of_table > 0:
    if test_data.values[i,1] == table.values[0,0]:
        table = table.drop(table.index[[0]])
        size_of_table = len(table)
        i+=1
    else:
        test_data = test_data.drop(test_data.index[[i]])

    if size_of_table == 0:
        k = i

        while len(test_data) > i:
            test_data = test_data.drop(test_data.index[[k]])

In [ ]:
test_data.head()

In [ ]:
test_data = test_data[['14. Depth (top reservoir ft TVD)', '17. Thickness (gross average ft)', '18. Thickness (net pay average ft)', '19. Porosity (matrix average %)',	'20. Permeability (air average mD)', '13. Structural setting', '15. Reservoir period', '16. Lithology (main)', '6. Tectonic regime', '11. Hydrocarbon type (main)']]

test_data.head()

In [ ]:
test_data = test_data.drop(columns=['19. Porosity (matrix average %)','6. Tectonic regime'])

In [ ]:
for column in ['13. Structural setting', '15. Reservoir period', '16. Lithology (main)','11. Hydrocarbon type (main)']:
     lee = preprocessing.LabelEncoder()
     test_data[column] = lee.fit_transform(test_data[column].astype(str))

test_data.head()

Если установить стратегию на 'kmeans', то предсказание '6. Tectonic regime становится точнее'.

Если установить стратегию на 'quantile', то предсказание '19. Porosity (matrix average %)' становится точнее.

In [ ]:
testformed_data = test_data.copy()
est = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
data_discrete = est.fit_transform(test_data.values[:,[0,1,2,3]])
testformed_data[['14. Depth (top reservoir ft TVD)', '17. Thickness (gross average ft)', '18. Thickness (net pay average ft)', '20. Permeability (air average mD)']] = data_discrete

In [ ]:
testformed_data.head()

Составление тренировочной выборки.

In [ ]:
train_data = geo_data.copy()
train_data = train_data.dropna()
train_data.head()

In [ ]:
train_data = train_data[['14. Depth (top reservoir ft TVD)', '17. Thickness (gross average ft)', '18. Thickness (net pay average ft)', '19. Porosity (matrix average %)', '20. Permeability (air average mD)', '13. Structural setting', '15. Reservoir period', '16. Lithology (main)', '6. Tectonic regime', '11. Hydrocarbon type (main)']]

train_data.head()

In [ ]:
leun = 0
for column in ['13. Structural setting', '15. Reservoir period', '16. Lithology (main)', '6. Tectonic regime', '11. Hydrocarbon type (main)']:
     le = preprocessing.LabelEncoder()
     if column == '6. Tectonic regime':
          leun = le
     train_data[column] = le.fit_transform(train_data[column].astype(str))

train_data.head()

In [ ]:
trainsformed_data = train_data.copy()
est = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
data_discrete = est.fit_transform(train_data.values[:,0:5])
trainsformed_data[['14. Depth (top reservoir ft TVD)', '17. Thickness (gross average ft)', '18. Thickness (net pay average ft)', '19. Porosity (matrix average %)', '20. Permeability (air average mD)']] = data_discrete

In [ ]:
trainsformed_data.head()

Основными параметрами выборок были выбраны:
- Depth - Глубина -
- Gross - Плотность -
- Net pay
- Porosity - Пористость -
- Permeability - Проницаемость -
- Structural setting - Структурная установка
- Reservoir period - Период резервуара
- Lithology - Литология
- Tectonic regime - Тектонический режим
- Hydrocarbon type - Углеводородный тип

<h2>Построение корреляционной матрицы</h2>

In [ ]:
#отбор столбцов для построения корреляционной матрицы
x_train = trainsformed_data

#построение диагональной маски
mask = np.zeros_like(x_train.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

f, ax = plt.subplots(figsize=(10, 10))

sns.heatmap(x_train.corr(), mask = mask, annot=True, fmt= '.1f', ax = ax, cmap = 'Blues')

<h2>Hill Climb Search</h2>

В сравнении метода hill climbing search исользуются методы оценки:

<h3>K2 Score</h3>

In [ ]:
hc1 = HillClimbSearch(trainsformed_data, scoring_method=K2Score(trainsformed_data))

<h3>BDeu Score</h3>

In [ ]:
hc2 = HillClimbSearch(trainsformed_data, scoring_method=BDeuScore(trainsformed_data))

<h3>Bic Score</h3>

In [ ]:
hc3 = HillClimbSearch(trainsformed_data, scoring_method=BicScore(trainsformed_data))

<h2>Оценка структуры данных</h2>

In [ ]:
tic = time.time()
model1 = hc1.estimate()
print('\n Время выполнения: ', time.time()-tic)

In [ ]:
tic = time.time()
model2 = hc2.estimate()
print('\n Время выполнения: ', time.time()-tic)

In [ ]:
tic = time.time()
model3 = hc3.estimate()
print('\n Время выполнения: ', time.time()-tic)

<h2>Проверка построенных структур данных</h2>

Как выяснилось, они одинаковые :)
<h2>Должны быть разные</h2>

In [ ]:
G_K1 = nx.DiGraph()
G_K1.add_edges_from(model1.edges())
pos = nx.layout.circular_layout(G_K1)
nx.draw(G_K1, pos, node_size=60, font_size=6, with_labels=True,font_weight='bold')

In [ ]:
G_K2 = nx.DiGraph()
G_K2.add_edges_from(model2.edges())
pos = nx.layout.circular_layout(G_K2)
nx.draw(G_K2, pos, node_size=5, font_size=7, with_labels=True,font_weight='bold')

In [ ]:
G_K3 = nx.DiGraph()
G_K3.add_edges_from(model3.edges())
pos = nx.layout.circular_layout(G_K3)
nx.draw(G_K3, pos, node_size=60, font_size=6, with_labels=True,font_weight='bold')

In [ ]:
# Создание байесовской модели.
bn1 = BayesianModel(model1.edges())

In [ ]:
# Создание байесовской модели.
bn2 = BayesianModel(model2.edges())

In [ ]:
# Создание байесовской модели.
bn3 = BayesianModel(model3.edges())

<h2>Обучение байесовских сетей</h2>

In [ ]:
tic = time.time()
# Обучение байесовской сети.
bn1.fit(trainsformed_data)
print('\n Время выполнения: ', time.time()-tic)

In [ ]:
tic = time.time()
# Обучение байесовской сети.
bn2.fit(trainsformed_data)
print('\n Время выполнения: ', time.time()-tic)

In [ ]:
tic = time.time()
# Обучение байесовской сети.
bn3.fit(trainsformed_data)
print('\n Время выполнения: ', time.time()-tic)

<h2>Заполнение пропусков</h2>

In [ ]:
tic = time.time()
# Заполнение пропусков
result_data1 = bn1.predict(testformed_data)
print('\n Время выполнения: ', time.time()-tic)

In [ ]:
tic = time.time()
# Заполнение пропусков
result_data2 = bn2.predict(testformed_data)
print('\n Время выполнения: ', time.time()-tic)

In [ ]:
tic = time.time()
# Заполнение пропусков
result_data3 = bn3.predict(testformed_data)
print('\n Время выполнения: ', time.time()-tic)

<h2>Проверка предсказанных значений</h2>

In [ ]:
result_data1.head()

In [ ]:
result_data2.head()

In [ ]:
result_data3.head()

<h2>Подсчет точности заполнения пропусков сетями</h2>

In [ ]:
result_data1.reset_index(drop=True, inplace=True)
testformed_data.reset_index(drop=True, inplace=True)
concat_data = result_data1[['19. Porosity (matrix average %)']]
concat_data = pd.concat([testformed_data[['14. Depth (top reservoir ft TVD)', '17. Thickness (gross average ft)', '18. Thickness (net pay average ft)', '20. Permeability (air average mD)']], concat_data],axis=1)

In [ ]:
concat_data.head()

In [ ]:
porosity = est.inverse_transform(concat_data)

In [ ]:
print(porosity)

In [ ]:
error = 0
for i in range(len(porosity)):
    print(porosity[i,4], mis_data.values[i,4])
    error += (porosity[i,4]-mis_data.values[i,4])**2

error = error / len(porosity)
print('\n Средневкадратическая ошибка: ', error)

In [ ]:
result_data2.reset_index(drop=True, inplace=True)
concat_data = result_data2[['19. Porosity (matrix average %)']]
concat_data = pd.concat([testformed_data[['14. Depth (top reservoir ft TVD)', '17. Thickness (gross average ft)', '18. Thickness (net pay average ft)', '20. Permeability (air average mD)']], concat_data],axis=1)

In [ ]:
porosity = est.inverse_transform(concat_data)

In [ ]:
error = 0
for i in range(len(porosity)):
    print(porosity[i,4], mis_data.values[i,4])
    error += (porosity[i,4]-mis_data.values[i,4])**2

error = error / len(porosity)
print('\n Средневкадратическая ошибка: ', error)

In [ ]:
result_data3.reset_index(drop=True, inplace=True)
concat_data = result_data3[['19. Porosity (matrix average %)']]
concat_data = pd.concat([testformed_data[['14. Depth (top reservoir ft TVD)', '17. Thickness (gross average ft)', '18. Thickness (net pay average ft)', '20. Permeability (air average mD)']], concat_data],axis=1)

In [ ]:
porosity = est.inverse_transform(concat_data)

In [ ]:
error = 0
for i in range(len(porosity)):
    print(porosity[i,4], mis_data.values[i,4])
    error += (porosity[i,4]-mis_data.values[i,4])**2

error = error / len(porosity)
print('\n Средневкадратическая ошибка: ', error)

In [ ]:
regime1 = leun.inverse_transform(result_data1.values[:,1].astype(int))

In [ ]:
regime2 = leun.inverse_transform(result_data2.values[:,1].astype(int))

In [ ]:
regime3 = leun.inverse_transform(result_data3.values[:,1].astype(int))

In [ ]:
def true_false(mis_data: pd.DataFrame, regime: pd.DataFrame):
    true, false = 0, 0

    for i in range(len(regime)):
        print(regime[i], mis_data.values[i,9], regime[i] in mis_data.values[i,9])
        if regime[i] in mis_data.values[i,9]:
            true += 1
        else:
            false += 1
    
    print('Число совпадений: ', true)
    print('Число промахов: ', false)
    
    ax = sns.barplot(x=['True','False'], y=[true, false])
    ax.set(xlabel='Значения', ylabel='n')

    plt.legend(handles=[true, false])
    plt.show()

In [ ]:
true_false(mis_data,regime1)

In [ ]:
true_false(mis_data,regime2)

In [ ]:
true_false(mis_data,regime3)

<h2>Нахождение среднего, максимального и минимального времен построения модели</h2>

In [ ]:
# md_times = []
# max_times = []
# min_times = []
# bn_times = []

# for i in range(3):
#      bn_times.append([])

# for i in range(100):
#      tic = time.time()
#      bn1.fit(trainsformed_data)
#      bn_times[0].append(time.time() - tic)
#      tic = time.time()
#      bn2.fit(trainsformed_data)
#      bn_times[1].append(time.time() - tic)
#      tic = time.time()
#      bn3.fit(trainsformed_data)
#      bn_times[2].append(time.time() - tic)

# for i in range(3):
#      max_times.append(max(bn_times[i]))
#      min_times.append(min(bn_times[i]))
#      md_times.append(sum(bn_times[i])/len(bn_times[i]))

In [ ]:
# def give_barplot(var):
#      sns.barplot(x=['K2Score','BDeuScore','BicScore'], y=var)
#      plt.show()

In [ ]:
# give_barplot(max_times)

In [ ]:
# give_barplot(min_times)

In [ ]:
# give_barplot(md_times)

<h2>PC algorithm</h2>

Не работает, потому что при построении модели куда-то теряется параметр :(

<h2>Создание модели</h2>.

In [ ]:
pc = PC(trainsformed_data)

In [ ]:
tic = time.time()
model = pc.estimate()
print('\n Время выполнения: ', time.time()-tic)

<h2>Оценка структуры данных</h2>

In [ ]:
G_K2 = nx.DiGraph()
G_K2.add_edges_from(model.edges())
pos = nx.layout.circular_layout(G_K2)
nx.draw(G_K2, pos, node_size=10, font_size=7, with_labels=True,font_weight='bold')

<h2>Создание байесовской сети</h2>

In [ ]:
# Создание байесовской модели.
bn = BayesianModel(model.edges())

<h2>Обучение сетей</h2>

In [ ]:
tic = time.time()
# Обучение параметров.
bn.fit(trainsformed_data)
print('\n Время выполнения: ', time.time()-tic)

In [ ]:
tic = time.time()
# Заполнение пропусков
result_data = bn.predict(testformed_data)
print('\n Время выполнения: ', time.time()-tic)

<h2>Подсчет точности заполнения пропусков сетями</h2>

In [ ]:
result_data.reset_index(drop=True, inplace=True)
concat_data = result_data[['19. Porosity (matrix average %)']]
concat_data = pd.concat([testformed_data[['14. Depth (top reservoir ft TVD)', '17. Thickness (gross average ft)', '18. Thickness (net pay average ft)', '20. Permeability (air average mD)']], concat_data],axis=1)

In [ ]:
porosity = est.inverse_transform(concat_data)

In [ ]:
error = 0
for i in range(len(porosity)):
     print(porosity[i,4], mis_data.values[i,4])
     error += (porosity[i,4]-mis_data.values[i,4])**2

error = error / len(porosity)
print('\n Средневкадратическая ошибка: ', error)

In [ ]:
# regime2 = leun.inverse_transform(result_data2.values[:,1].astype(int))

In [ ]:
true_false(mis_data,regime2)

<h2>Tree search</h2>

In [ ]:
from pgmpy.estimators import TreeSearch
best = TreeSearch(trainsformed_data, root_node='19. Porosity (matrix average %)')
model = best.estimate(estimator_type='chow-liu')

In [ ]:
G_K2 = nx.DiGraph()
G_K2.add_edges_from(model.edges())
pos = nx.layout.circular_layout(G_K2)
nx.draw(G_K2, pos, node_size=60, font_size=6, with_labels=True,font_weight='bold')

<h2>Создание байесовской сети</h2>

In [ ]:
# Создание байесовской модели.
bn = BayesianModel(model.edges())

In [ ]:
tic = time.time()
# Обучение байесовской сети.
bn.fit(trainsformed_data)
print('\n Время выполнения: ', time.time()-tic)

In [ ]:
tic = time.time()
# Заполнение пропусков
result_data = bn.predict(testformed_data)
print('\n Время выполнения: ', time.time()-tic)

In [ ]:
result_data.head()

In [ ]:
result_data.reset_index(drop=True, inplace=True)
testformed_data.reset_index(drop=True, inplace=True)
concat_data = result_data1[['19. Porosity (matrix average %)']]
concat_data = pd.concat([testformed_data[['14. Depth (top reservoir ft TVD)', '17. Thickness (gross average ft)', '18. Thickness (net pay average ft)', '20. Permeability (air average mD)']], concat_data],axis=1)

In [ ]:
concat_data.head()

In [ ]:
porosity = est.inverse_transform(concat_data)

In [ ]:


error = 0
for i in range(len(porosity)):
    print(porosity[i,4], mis_data.values[i,4])
    error += abs(porosity[i,4]-mis_data.values[i,4])

error = error / len(porosity)
print('\n Средняя ошибка: ', error)

In [ ]:
regime = leun.inverse_transform(result_data.values[:,1].astype(int))

In [ ]:
true_false(mis_data,regime)

Посмотреть реализации для гетерогенных сетей.